In [ ]:
import os
import sys
from typing import Union, Any, Optional, Callable, Tuple

ROOT_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
if ROOT_DIR not in sys.path: sys.path.append(ROOT_DIR)

import numpy as np
import pandas as pd
import proplot as plot
import eagerpy as ep
import torch

from DeepSparseCoding.tf1x.utils.logger import Logger as tfLogger
import DeepSparseCoding.tf1x.analysis.analysis_picker as ap
from DeepSparseCoding.tf1x.data.dataset import Dataset
import DeepSparseCoding.tf1x.utils.data_processing as tfdp

from DeepSparseCoding.utils.file_utils import Logger
import DeepSparseCoding.utils.dataset_utils as dataset_utils
import DeepSparseCoding.utils.loaders as loaders
import DeepSparseCoding.utils.plot_functions as pf

from foolbox import PyTorchModel, accuracy
from foolbox.attacks.projected_gradient_descent import LinfProjectedGradientDescentAttack
from foolbox.types import Bounds
from foolbox.models.base import Model
from foolbox.attacks.base import T
from foolbox.criteria import Misclassification
from foolbox.attacks.base import raise_if_kwargs
from foolbox.attacks.base import get_criterion

rand_state = np.random.RandomState(123)

### Load Foolbox Pytorch models & data

In [ ]:
def create_mnist_dsc(log_file, cp_file):
    logger = Logger(log_file, overwrite=False)
    log_text = logger.load_file()
    params = logger.read_params(log_text)[-1]
    params.cp_latest_filename = cp_file
    params.standardize_data = False
    params.rescale_data_to_one = True
    params.shuffle_data = False
    train_loader, val_loader, test_loader, data_params = dataset_utils.load_dataset(params)
    for key, value in data_params.items():
        setattr(params, key, value)
    model = loaders.load_model(params.model_type)
    model.setup(params, logger)
    model.params.analysis_out_dir = os.path.join(
        *[model.params.model_out_dir, 'analysis', model.params.version])
    model.params.analysis_save_dir = os.path.join(model.params.analysis_out_dir, 'savefiles')
    if not os.path.exists(model.params.analysis_save_dir):
        os.makedirs(model.params.analysis_save_dir)
    model.to(params.device)
    model.load_checkpoint()
    fmodel = PyTorchModel(model.eval(), bounds=(0, 1))
    return fmodel, model, test_loader, model.params.batch_size, model.params.device

log_files = [
    os.path.join(*[ROOT_DIR, 'Torch_projects', 'mlp_768_mnist', 'logfiles', 'mlp_768_mnist_v0.log']),
    os.path.join(*[ROOT_DIR, 'Torch_projects', 'lca_768_mlp_mnist', 'logfiles', 'lca_768_mlp_mnist_v0.log'])
]

cp_latest_filenames = [
    os.path.join(*[ROOT_DIR,'Torch_projects', 'mlp_768_mnist', 'checkpoints', 'mlp_768_mnist_latest_checkpoint_v0.pt']),
    os.path.join(*[ROOT_DIR, 'Torch_projects', 'lca_768_mlp_mnist', 'checkpoints', 'lca_768_mlp_mnist_latest_checkpoint_v0.pt'])
]

fmodel_mlp, dsc_model_mlp, test_loader, batch_size, device = create_mnist_dsc(log_files[0], cp_latest_filenames[0])
fmodel_mlp.model_type = 'MLP'

fmodel_lca, dsc_model_lca = create_mnist_dsc(log_files[1], cp_latest_filenames[1])[:2]
fmodel_lca.model_type = 'LCA'

fmodels = [fmodel_mlp, fmodel_lca]

fb_image_batch, fb_label_batch = next(iter(test_loader))
fb_image_batch = fb_image_batch.reshape((batch_size, 784))

### Load DeepSparseCoding Tensorflow models & data

In [ ]:
class params(object):
  def __init__(self):
    self.device = "/gpu:0"
    self.analysis_dataset = "test"
    self.save_info = "analysis_" + self.analysis_dataset
    self.overwrite_analysis_log = False
    self.do_class_adversaries = True
    self.do_run_analysis = False
    self.do_evals = False
    self.do_basis_analysis = False
    self.do_inference = False
    self.do_atas = False 
    self.do_recon_adversaries = False
    self.do_neuron_visualization = False
    self.do_full_recon = False
    self.do_orientation_analysis = False 
    self.do_group_recons = False
    
    #Adversarial params
    self.adversarial_attack_method = "kurakin_untargeted"
    self.adversarial_step_size = 0.005 # learning rate for optimizer
    self.adversarial_num_steps = 500 # Number of iterations adversarial attacks
    self.confidence_threshold = 0.9
    self.adversarial_max_change = 1.0 # maximum size of adversarial perturation (epsilon)
    self.carlini_change_variable = False # whether to use the change of variable trick from carlini et al
    self.adv_optimizer = "sgd" # attack optimizer
    self.adversarial_target_method = "random" # Not used if attack_method is untargeted#TODO support specified
    self.adversarial_clip = True # whether or not to clip the final perturbed image
    self.adversarial_clip_range = [0.0, 1.0] # Maximum range of image values
    #self.carlini_recon_mult = 0.1#list(np.arange(.5, 1, .1))
    self.adversarial_save_int = 1 # Interval at which to save adv examples to the npz file
    self.eval_batch_size = 50 # batch size for computing adv examples
    self.adversarial_input_id = np.arange(self.eval_batch_size, dtype=np.int32) # Which adv images to use; None to use all
    self.adversarial_target_labels = None # Parameter for "specified" target_method. Only for class attacks. Needs to be a list or numpy array of size [adv_batch_size]

In [ ]:
analysis_params = params()
analysis_params.projects_dir = os.path.expanduser("~")+"/Work/Projects/"

#model_names = ['mlp_lca_768_latent_75_steps_mnist', 'slp_lca_768_latent_75_steps_mnist']
model_names = ['mlp_cosyne_mnist', 'slp_lca_768_latent_75_steps_mnist']
model_types = ['MLP', 'LCA']
analyzers = []
for model_type, model_name in zip(model_types, model_names):
    analysis_params.model_name = model_name
    analysis_params.version = '0.0'
    analysis_params.model_dir = analysis_params.projects_dir+analysis_params.model_name
    model_log_file = (analysis_params.model_dir+"/logfiles/"+analysis_params.model_name
      +"_v"+analysis_params.version+".log")
    model_logger = tfLogger(model_log_file, overwrite=False)
    model_log_text = model_logger.load_file()
    model_params = model_logger.read_params(model_log_text)[-1]
    analysis_params.model_type = model_params.model_type
    analyzer = ap.get_analyzer(analysis_params.model_type)
    analysis_params.save_info = "analysis_test_" + analysis_params.analysis_dataset
    analysis_params.save_info += (
        "_linf_"+str(analysis_params.adversarial_max_change)
        +"_ss_"+str(analysis_params.adversarial_step_size)
        +"_ns_"+str(analysis_params.adversarial_num_steps)
        +"_pgd_untargeted"
    )
    analyzer.setup(analysis_params)
    analyzer.model_type = model_type
    analyzer.confidence_threshold = analysis_params.confidence_threshold
    analyzers.append(analyzer)

mnist_data = test_loader.dataset.data.numpy().astype(np.float32)
mnist_data /= 255
dsc_data = {
    'test':Dataset(
        np.expand_dims(mnist_data, axis=-1),
        tfdp.dense_to_one_hot(test_loader.dataset.targets.numpy(), 10),
        None,
        rand_state
    )
}
dsc_data = analyzers[0].model.reshape_dataset(dsc_data, analyzer.model_params)
for analyzer in analyzers:
    analyzer.model_params.data_shape = list(dsc_data["test"].shape[1:])
    analyzer.setup_model(analyzer.model_params)
dsc_image_batch, dsc_label_batch, _ = dsc_data['test'].next_batch(batch_size, shuffle_data=False)

### Compare DeepSparseCoding & Foolbox data & logits

In [ ]:
img_idx = np.random.randint(batch_size)
fig, axs = plot.subplots(ncols=3)
im = axs[0].imshow(fb_image_batch.numpy()[img_idx,...].reshape(28, 28), cmap='grays_r')
axs[0].format(title=f'PyTorch loader digit class {fb_label_batch[img_idx]}')
axs[0].colorbar(im)
im = axs[1].imshow(dsc_image_batch[img_idx,...].reshape(28, 28), cmap='grays_r')
axs[1].format(title=f'DSC dataset digit class {tfdp.one_hot_to_dense(dsc_label_batch)[img_idx]}')
axs[1].colorbar(im)
diff_img = np.abs(dsc_image_batch[img_idx,...].reshape(28, 28) - fb_image_batch.numpy()[img_idx,...].reshape(28, 28))
im = axs[2].imshow(diff_img, cmap='grays_r')
axs[2].format(title=f'Difference image')
axs[2].colorbar(im)
pf.clear_axes(axs)
plot.show()

In [ ]:
dsc_test_image = dsc_image_batch[img_idx, ...][None, :]
fb_test_image = fb_image_batch[img_idx, ...][None, :]
dsc_forward = [np.squeeze(analyzer.compute_activations(dsc_test_image,
    activation_operation=analyzer.model.get_logits))
    for analyzer in analyzers]
fb_forward = [fmodel(fb_test_image.to(device)).cpu().numpy() for fmodel in fmodels]
fig, axs = plot.subplots(ncols=2, nrows=2)
axs[0,0].bar(np.squeeze(dsc_forward[0]))
axs[0,0].format(title=f'DSC_{analyzers[0].model_type}')
axs[0,1].bar(np.squeeze(dsc_forward[1]))
axs[0,1].format(title=f'DSC_{analyzers[1].model_type}')
axs[1,0].bar(np.squeeze(fb_forward[0]))
axs[1,0].format(title=f'FB_{fmodels[0].model_type}')
axs[1,1].bar(np.squeeze(fb_forward[1]))
axs[1,1].format(title=f'FB_{fmodels[1].model_type}')
axs.format(xminorlocator='null', xlocator=('linear', 10)) # TODO: Broken?
plot.show()

In [ ]:
dsc_test_image = dsc_image_batch#[img_idx, ...][None, :]
fb_test_image = fb_image_batch#[img_idx, ...][None, :]

dsc_forward = [np.squeeze(analyzer.compute_activations(dsc_test_image,
    activation_operation=analyzer.model.get_label_est))
    for analyzer in analyzers]
dsc_forward = np.stack(dsc_forward, axis=0)

fb_forward = [torch.nn.functional.softmax(fmodel(fb_test_image.to(device)), dim=-1).cpu().numpy() for fmodel in fmodels]
fb_forward = np.stack(fb_forward, axis=0)

all_results = np.concatenate((dsc_forward.reshape( 2, -1), fb_forward.reshape(2, -1)), axis=0).T

In [ ]:
fig, axs = plot.subplots(ncols=2, nrows=2)
axs[0,0].bar(np.squeeze(dsc_forward[0, img_idx]))
axs[0,0].format(title=f'DSC_{analyzers[0].model_type}')
axs[0,1].bar(np.squeeze(dsc_forward[1, img_idx]))
axs[0,1].format(title=f'DSC_{analyzers[1].model_type}')
axs[1,0].bar(np.squeeze(fb_forward[0, img_idx]))
axs[1,0].format(title=f'FB_{fmodels[0].model_type}')
axs[1,1].bar(np.squeeze(fb_forward[1, img_idx]))
axs[1,1].format(title=f'FB_{fmodels[1].model_type}')
axs.format(suptitle='Softmax Confidence', xminorlocator='null', xlocator=('linear', 10), ylim=[0, 1])
plot.show()

In [ ]:
names = ['DSC_MLP', 'DSC_LCA', 'FB_MLP', 'FB_LCA']
data = pd.DataFrame(
    all_results,
    columns=pd.Index(names, name='Model')
)

fig, ax = plot.subplots(ncols=1, axwidth=2.5, share=0)
ax.format(grid=False, suptitle='L infinity Attack Mean Squared Distances')
obj1 = ax.boxplot(
    data, linewidth=0.7, marker='.', fillcolor='gray5',
    medianlw=1, mediancolor='k', meancolor='k', meanlw=1
)
ax.format(title='Test set confidences')

### Run DeepSparseCoding adversarial attack

In [ ]:
def get_keep_and_confidence_indices(softmax_conf, all_kept_indices, confidence_threshold, num_images, labels):
    softmax_conf[np.arange(num_images, dtype=np.int32), labels] = 0 # zero confidence at true label
    confidence_indices = np.max(softmax_conf, axis=-1) # highest non-true label confidence
    all_above_thresh = np.nonzero(np.squeeze(confidence_indices>confidence_threshold))[0]
    keep_indices = np.array([], dtype=np.int32)
    for adv_index in all_above_thresh:
        if adv_index not in set(all_kept_indices):
            keep_indices = np.append(keep_indices, adv_index)
    return keep_indices, confidence_indices

In [ ]:
for analyzer in analyzers:
    analyzer.class_adversary_analysis(dsc_image_batch,
        dsc_label_batch,
        batch_size=analyzer.analysis_params.eval_batch_size,
        input_id=analyzer.analysis_params.adversarial_input_id,
        target_method = analyzer.analysis_params.adversarial_target_method,
        target_labels = analyzer.analysis_params.adversarial_target_labels,
        save_info=analyzer.analysis_params.save_info)

In [ ]:
data = dsc_image_batch
labels = tfdp.one_hot_to_dense(dsc_label_batch.astype(np.int32))
for analyzer in analyzers:
    store_data = np.zeros_like(data)
    store_time_step = -1*np.ones(data.shape[0], dtype=np.int32)
    store_confidence = np.zeros(data.shape[0], dtype=np.float32)
    store_mses = np.zeros(data.shape[0], dtype=np.float32)
    all_kept_indices = []
    for adv_step in range(1, analyzer.analysis_params.adversarial_num_steps+1): # first one is original
        keep_indices, confidence_indices = get_keep_and_confidence_indices(
            analyzer.adversarial_outputs[0, adv_step, ...],
            all_kept_indices,
            analyzer.confidence_threshold,
            data.shape[0],
            labels)
        if keep_indices.size > 0:
            all_kept_indices.extend(keep_indices)
            store_data[keep_indices, ...] = analyzer.adversarial_images[0, adv_step, keep_indices, ...]
            store_time_step[keep_indices] = adv_step
            store_confidence[keep_indices] = confidence_indices[keep_indices]
            store_mses[keep_indices] = analyzer.adversarial_input_adv_mses[0, adv_step, keep_indices]
    batch_indices = np.arange(data.shape[0], dtype=np.int32)[:,None]
    failed_indices = np.array([val for val in batch_indices if val not in all_kept_indices])
    if len(failed_indices) > 0:
        store_confidence[failed_indices] = confidence_indices[failed_indices]
        store_data[failed_indices, ...] = data[failed_indices, ...]
        store_mses[failed_indices] = analyzer.adversarial_input_adv_mses[0, -1, failed_indices]
    analyzer.adversarial_images = store_data
    analyzer.adversarial_time_step = store_time_step
    analyzer.adversarial_confidence = store_confidence
    analyzer.failed_indices = failed_indices
    analyzer.success_indices = list(set(all_kept_indices))
    analyzer.mean_squared_distances = store_mses
    analyzer.num_failed = data.shape[0] - len(set(all_kept_indices))
    print(f'model {analyzer.model_type} had {analyzer.num_failed} failed indices')

### Run Foolbox adversarial attack

In [ ]:
class LinfProjectedGradientDescentAttackWithStopping(LinfProjectedGradientDescentAttack):
    def __init__(
        self,
        *,
        rel_stepsize: float = 0.025,
        abs_stepsize: Optional[float] = None,
        steps: int = 50,
        random_start: bool = True,
    ):
        super().__init__(
            rel_stepsize=rel_stepsize,
            abs_stepsize=abs_stepsize,
            steps=steps,
            random_start=random_start,
        )
        
    #def project(self, x: ep.Tensor, x0: ep.Tensor, epsilon: float) -> ep.Tensor:
    #    return x0 + ep.clip(x - x0, -epsilon, epsilon)
    
    def normalize(
        self, gradients: ep.Tensor, *, x: ep.Tensor, bounds: Bounds
    ) -> ep.Tensor:
        return gradients.sign()
        
    def run(
        self,
        model: Model,
        inputs: T,
        criterion: Union[Misclassification, T],
        *,
        epsilon: float,
        **kwargs: Any,
    ) -> T:
        raise_if_kwargs(kwargs)
        x0, restore_type = ep.astensor_(inputs)
        criterion_ = get_criterion(criterion)
        del inputs, criterion, kwargs

        if not isinstance(criterion_, Misclassification):
            raise ValueError("unsupported criterion")

        labels = criterion_.labels
        loss_fn = self.get_loss_fn(model, labels)

        if self.abs_stepsize is None:
            stepsize = self.rel_stepsize * epsilon
        else:
            stepsize = self.abs_stepsize

        orig_x = x0.numpy().copy()
        x = x0

        if self.random_start:
            x = self.get_random_start(x0, epsilon)
            x = ep.clip(x, *model.bounds)
        else:
            x = x0
        store_x = np.zeros_like(x)
        store_time_step = -1*np.ones(x.shape[0], dtype=np.int32)
        store_confidence = np.zeros(x.shape[0], dtype=np.float32)
        all_kept_indices = []
        time_step = 0
        num_failed = 0
        while len(set(all_kept_indices)) < x.shape[0]:
            loss, gradients = self.value_and_grad(loss_fn, x)
            gradients = self.normalize(gradients=gradients, x=x, bounds=model.bounds)
            x = x + stepsize * gradients
            x = self.project(x, x0, epsilon)
            x = ep.clip(x, *model.bounds)
            
            keep_indices, confidence_indices = get_keep_and_confidence_indices(
                ep.softmax(model(x)).numpy().copy(),
                all_kept_indices,
                model.confidence_threshold,
                x.shape[0],
                labels.numpy())
            if keep_indices.size > 0:
                all_kept_indices.extend(keep_indices)
                store_x[keep_indices, ...] = x.numpy()[keep_indices, ...]
                store_time_step[keep_indices] = time_step
                store_confidence[keep_indices] = confidence_indices[keep_indices]
            time_step += 1
            if time_step == self.steps-1:
                num_failed = x.shape[0] - len(set(all_kept_indices))
                print(f'Max steps = {self.steps} reached for model {model.model_type}, {num_failed} images did not achieve adversarial confidence threshold of {model.confidence_threshold}')
                break
        batch_indices = np.arange(x.shape[0], dtype=np.int32)[:,None]
        failed_indices = np.array([val for val in batch_indices if val not in all_kept_indices])
        if len(failed_indices) > 0:
            store_confidence[failed_indices] = confidence_indices[failed_indices]
            store_x[failed_indices, ...] = x[failed_indices, ...]
        reduc_dim = tuple(range(1, len(orig_x.shape)))
        msd = np.mean((store_x - orig_x)**2, axis=reduc_dim)
        model.adversarial_images.append(store_x)
        model.adversarial_time_step.append(store_time_step)
        model.adversarial_confidence.append(store_confidence)
        model.success_indices.append(np.array(all_kept_indices, dtype=np.int32))
        model.failed_indices.append(failed_indices)
        model.mean_squared_distances.append(msd)
        model.num_failed.append(len(failed_indices))
        return restore_type(x)

In [ ]:
attack_params = {
    'LinfPGD': {
        'random_start':False,
        'abs_stepsize':analysis_params.adversarial_step_size,
        'steps':analysis_params.adversarial_num_steps # maximum number of steps
    }
}
epsilons = [analysis_params.adversarial_max_change]
attack = LinfProjectedGradientDescentAttackWithStopping(**attack_params['LinfPGD'])

for fmodel in fmodels:
    fmodel.confidence_threshold = analysis_params.confidence_threshold
    fmodel.adversarial_images = []
    fmodel.adversarial_time_step = []
    fmodel.adversarial_confidence = []
    fmodel.failed_indices = []
    fmodel.mean_squared_distances = []
    fmodel.num_failed = []
    fmodel.success_indices = []
    fmodel.success = []
    advs, _, success = attack(
        fmodel,
        fb_image_batch.to(device),
        fb_label_batch.to(device),
        epsilons=epsilons
    )
    fmodel.success.append(success.type(torch.float32).cpu().numpy())
    print(f'model {fmodel.model_type} had {fmodel.num_failed} failed indices')

### Compare DeepSparseCoding & Foolbox adversarial attacks

In [ ]:
for analyzer in analyzers:
    analyzer.confidence = analyzer.adversarial_outputs[0, 0, ...]
    analyzer.accuracy = analyzer.adversarial_clean_accuracy.item()
    print(f'DSC {analyzer.model_type} clean accuracy = {analyzer.accuracy} and adv accuracy = {analyzer.adversarial_adv_accuracy}')
    
for fmodel in fmodels:
    logits = fmodel(fb_image_batch.to(device))
    confidence = torch.nn.functional.softmax(logits, dim=-1)
    fmodel.confidence = confidence.cpu().numpy()
    fmodel.accuracy = accuracy(fmodel, fb_image_batch.to(device), fb_label_batch.to(device))
    print(f'FB {fmodel.model_type} clean accuracy = {fmodel.accuracy} and adv accuracy = {1.0 - fmodel.success[0].mean(axis=-1).round(2)}')

In [ ]:
num_bins = 100
fig, axs = plot.subplots(ncols=2, nrows=2)
for ax, model, atk_type in zip(axs, analyzers+fmodels, ['DSC', 'DSC', 'FB', 'FB']):
    max_confidence = np.max(model.confidence, axis=1) # max is across categories, per image
    bins = np.linspace(0, 1, num_bins)
    count, bin_edges = np.histogram(max_confidence, bins)
    bin_left, bin_right = bin_edges[:-1], bin_edges[1:]
    bin_centers = bin_left + (bin_right - bin_left)/2
    ax.bar(bin_centers, count, width=1, color='k')
    mean_confidence = np.mean(max_confidence)
    mean_idx = np.abs(bin_edges - mean_confidence).argmin()
    mean_conf_bin = bin_edges[mean_idx]
    ax.axvline(mean_conf_bin, lw=2, ls='-', color='r')
    ax.format(
        title=f'{atk_type}_{model.model_type}\nMean confidence = {mean_conf_bin}',
        xlim=[0, 1.0]
    )
axs.format(
    suptitle='Softmax confidence on clean images',
    ylabel='Count',
    xlabel='Confidence'
)

# TODO: What's up with this plot? shouldn't have a plateu

In [ ]:
num_bins = 50
bins = np.linspace(0, analysis_params.adversarial_num_steps, num_bins)

fig, axs = plot.subplots(ncols=2, nrows=2)
for ax, model, atk_type in zip(axs, analyzers+fmodels, ['DSC', 'DSC', 'FB', 'FB']):
    count, bin_edges = np.histogram(model.success_indices, bins)
    bin_left, bin_right = bin_edges[:-1], bin_edges[1:]
    bin_centers = bin_left + (bin_right - bin_left)/2
    ax.bar(bin_centers, count, width=1, color='k')
    mean_idx = np.mean(model.success_indices)
    mean_success_idx = np.abs(model.success_indices - mean_idx).argmin()
    mean_bin_idx = np.abs(bin_edges - mean_idx).argmin()
    mean_success_bin = bin_edges[mean_bin_idx]
    ax.axvline(mean_success_bin, lw=1, ls='-', color='r')
    ax.format(title=f'{atk_type}_{model.model_type}\nMean success timestep = {mean_success_idx}')

In [ ]:
names = ['MLP 2L;768N','LCA 2L;768N']

fb_all_success_indices = np.intersect1d(*[fmodel.success_indices[0] for fmodel in fmodels])
fb_adv_results_list = [np.array(fmodel.mean_squared_distances[0])[fb_all_success_indices] for fmodel in fmodels]
fb_all_results = np.stack(fb_adv_results_list, axis=-1).squeeze()
fb_data = pd.DataFrame(
    fb_all_results,
    columns=pd.Index(names, name='Model')
)

dsc_all_success_indices = np.intersect1d(*[analyzer.success_indices for analyzer in analyzers])
dsc_adv_results_list = [analyzer.mean_squared_distances[dsc_all_success_indices] for analyzer in analyzers]
dsc_all_results = np.stack(dsc_adv_results_list, axis=-1).squeeze()
dsc_data = pd.DataFrame(
    dsc_all_results,
    columns=pd.Index(names, name='Model')
)

fig, axs = plot.subplots(ncols=2, axwidth=2.5, share=0)
axs.format(grid=False, suptitle='L infinity Attack Mean Squared Distances')
ax = axs[0]
obj1 = ax.boxplot(
    fb_data, linewidth=0.7, marker='.', fillcolor='gray5',
    medianlw=1, mediancolor='k', meancolor='k', meanlw=1
)
ax.format(title='Foolbox')

ax = axs[1]
obj2 = ax.boxplot(
    dsc_data, linewidth=0.7, marker='.', fillcolor='gray5',
    medianlw=1, mediancolor='k', meancolor='k', meanlw=1
)
ax.format(title='Deep Sparse Coding')

## attack images